<a href="https://colab.research.google.com/github/anshmehta337/deep-learning/blob/main/CIFAR100.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import torch
import torch.optim as optim
import torch.nn as nn
import torchvision.transforms as transforms
import torchvision.datasets as dataset
from torch.utils.data import DataLoader

In [7]:
train_transform=transforms.Compose([
    transforms.RandomRotation(10),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [8]:
test_transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.5,0.5,0.5),(0.5,0.5,0.5))
])

In [9]:
train_dataset=dataset.CIFAR100(root='./data',train=True,download=True,transform=train_transform)
test_dataset=dataset.CIFAR100(root='./data',train=False,download=True,transform=test_transform)


In [10]:
train_loader=DataLoader(train_dataset,batch_size=128,shuffle=True,num_workers=4)

test_loader=DataLoader(test_dataset,batch_size=128,shuffle=False,num_workers=4)


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [11]:
#lets make resnet architecture

In [12]:
class build_res(nn.Module):
  def __init__(self,in_channels,out_channels,drop=0.0,down_sample=False):
    super().__init__()
    stride=2 if down_sample else 1
    self.conv1=nn.Conv2d(in_channels,out_channels,kernel_size=3,stride=stride,padding=1,bias=False)
    self.bn1=nn.BatchNorm2d(out_channels)
    self.relu=nn.ReLU(inplace=True)

    self.conv2=nn.Conv2d(out_channels,out_channels,kernel_size=3,stride=1,padding=1,bias=False)
    self.bn2=nn.BatchNorm2d(out_channels)
    self.drop=nn.Dropout(drop)

    self.shortcut=nn.Sequential()
    if in_channels!=out_channels or down_sample:
      self.shortcut=nn.Sequential(
          nn.Conv2d(in_channels,out_channels,kernel_size=1,stride=stride,bias=False),
          nn.BatchNorm2d(out_channels)
      )
  def forward(self,x):
      store=x
      x=self.conv1(x)
      x=self.bn1(x)
      x=self.relu(x)
      x=self.conv2(x)
      x=self.bn2(x)
      x=self.drop(x)
      x+=self.shortcut(store)
      x=self.relu(x)
      return x


In [13]:
class godeep(nn.Module):
  def __init__(self,n_class=100):
    super().__init__()
    self.block1=nn.Sequential(
        build_res(3,64,drop=0.2,down_sample=False),
        build_res(64,64,drop=0.2),
        build_res(64,64,drop=0.2),
    )
    self.block2=nn.Sequential(
        build_res(64,128,drop=0.2,down_sample=True),
        build_res(128,128,drop=0.2),
        build_res(128,128,drop=0.2),
        build_res(128,128,drop=0.2),
    )
    self.block3=nn.Sequential(
         build_res(128,256,drop=0.3,down_sample=True),
         build_res(256,256,drop=0.3),
         build_res(256,256,drop=0.3),
         build_res(256,256,drop=0.3),
         build_res(256,256,drop=0.3),
    )
    self.block4=nn.Sequential(
        build_res(256,512,drop=0.3,down_sample=True),
        build_res(512,512,drop=0.3),
        build_res(512,512,drop=0.3),
    )
    #pred now
    self.avgpool=nn.AdaptiveAvgPool2d((1,1))
    self.fc=nn.Sequential(
         nn.Flatten(),
            nn.Linear(512, 256),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(256, 128),
            nn.ReLU(),
            nn.Dropout(0.3),
            nn.Linear(128, 100)

    )
  def forward(self,x):
      x=self.block1(x)
      x=self.block2(x)
      x=self.block3(x)
      x=self.block4(x)
      x=self.avgpool(x)
      x=self.fc(x)
      return x


In [14]:
#now train our network
class EarlyStopping:
    def __init__(self, patience=10, min_delta=0):
        self.patience = patience
        self.min_delta = min_delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None:
            self.best_loss = val_loss
        elif val_loss > self.best_loss - self.min_delta:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_loss = val_loss
            self.counter = 0


In [15]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

model = godeep().to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=100)


In [16]:
def train_one_epoch(epoch):
    model.train()
    running_loss, correct, total = 0, 0, 0

    for inputs, targets in train_loader:
        inputs, targets = inputs.to(device), targets.to(device)

        optimizer.zero_grad()
        outputs = model(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        running_loss += loss.item() * inputs.size(0)
        _, predicted = outputs.max(1)
        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()

    print(f"Epoch {epoch} | Train Loss: {running_loss/total:.4f} | Train Acc: {100*correct/total:.2f}%")

def evaluate():
    model.eval()
    correct, total, loss_sum = 0, 0, 0
    with torch.no_grad():
        for inputs, targets in test_loader:
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = model(inputs)
            loss_sum += criterion(outputs, targets).item() * inputs.size(0)
            _, predicted = outputs.max(1)
            total += targets.size(0)
            correct += predicted.eq(targets).sum().item()
    return loss_sum / total, 100 * correct / total


early_stopping = EarlyStopping(patience=10, min_delta=0.001)

for epoch in range(100):
    train_one_epoch(epoch)
    val_loss, val_acc = evaluate()
    print(f"Validation Loss: {val_loss:.4f} | Validation Acc: {val_acc:.2f}%")

    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("Early stopping triggered!")
        break

    scheduler.step()


Epoch 0 | Train Loss: 4.3266 | Train Acc: 3.16%
Validation Loss: 4.0796 | Validation Acc: 5.01%
Epoch 1 | Train Loss: 4.0648 | Train Acc: 5.48%
Validation Loss: 3.9200 | Validation Acc: 7.58%
Epoch 2 | Train Loss: 3.8922 | Train Acc: 7.68%
Validation Loss: 3.7608 | Validation Acc: 10.50%
Epoch 3 | Train Loss: 3.7291 | Train Acc: 10.13%
Validation Loss: 3.5654 | Validation Acc: 13.34%
Epoch 4 | Train Loss: 3.5472 | Train Acc: 13.10%
Validation Loss: 3.2815 | Validation Acc: 17.93%
Epoch 5 | Train Loss: 3.3584 | Train Acc: 16.52%
Validation Loss: 3.0904 | Validation Acc: 21.13%
Epoch 6 | Train Loss: 3.1369 | Train Acc: 20.46%
Validation Loss: 2.8321 | Validation Acc: 26.05%
Epoch 7 | Train Loss: 2.9450 | Train Acc: 24.02%
Validation Loss: 2.8769 | Validation Acc: 25.70%
Epoch 8 | Train Loss: 2.7660 | Train Acc: 27.78%
Validation Loss: 2.7195 | Validation Acc: 30.11%
Epoch 9 | Train Loss: 2.6101 | Train Acc: 31.04%
Validation Loss: 2.9668 | Validation Acc: 27.94%
Epoch 10 | Train Loss: 2.